# 快速資料準備 - 多核加速版
## 針對人體姿勢辨識的熱成像超解析度資料準備

In [1]:
import json
import os
from pathlib import Path
from datetime import datetime, timedelta
from typing import List, Tuple, Dict, Optional
import warnings
warnings.filterwarnings('ignore')

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

# 多核處理
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from multiprocessing import cpu_count
import time

print(f"✓ Libraries loaded")
print(f"CPU cores available: {cpu_count()}")

✓ Libraries loaded
CPU cores available: 4


## 1. 設定

In [2]:
# ==================== 路徑 ====================
DATA_DIR = Path('./data')
THERMAL_DIR = DATA_DIR / 'thermal'
CAMERA_DIR = DATA_DIR / 'camera'

AVI_FILE = CAMERA_DIR / 'output.avi'
OUTPUT_TXT = CAMERA_DIR / 'output.txt'

OUTPUT_DIR = Path('./output')
SYNCHRONIZED_DIR = OUTPUT_DIR / 'synchronized_final'
RGB_EXPORT_DIR = OUTPUT_DIR / 'rgb_for_labeling'

for dir_path in [SYNCHRONIZED_DIR, RGB_EXPORT_DIR]:
    dir_path.mkdir(parents=True, exist_ok=True)
    (dir_path / 'thermal').mkdir(exist_ok=True)
    (dir_path / 'rgb').mkdir(exist_ok=True)

# ==================== 參數 ====================
THERMAL_RES = (32, 24)
TARGET_FPS = 8
FRAME_INTERVAL = 1.0 / TARGET_FPS
TIMESTAMP_TOLERANCE = FRAME_INTERVAL / 2

FLIP_THERMAL = True
FLIP_RGB = False

# 多核設定
NUM_WORKERS = max(1, cpu_count() - 1)  # 保留 1 個核心給系統
CHUNK_SIZE = 100  # 每次處理的批次大小

print(f"Using {NUM_WORKERS} CPU cores for parallel processing")

Using 3 CPU cores for parallel processing


## 2. 解析 output.txt (單核，很快)

In [3]:
import re

def parse_output_txt(filepath: Path) -> Optional[datetime]:
    if not filepath.exists():
        return None
    
    with open(filepath, 'r', encoding='utf-8') as f:
        content = f.read()
    
    patterns = [
        (r'(\d{4}-\d{2}-\d{2}\s+\d{2}:\d{2}:\d{2})', '%Y-%m-%d %H:%M:%S'),
        (r'(\d{4}/\d{2}/\d{2}\s+\d{2}:\d{2}:\d{2})', '%Y/%m/%d %H:%M:%S'),
        (r'(\d{10,})', 'unix'),
    ]
    
    for pattern, fmt in patterns:
        match = re.search(pattern, content)
        if match:
            time_str = match.group(1)
            try:
                if fmt == 'unix':
                    return datetime.fromtimestamp(int(time_str))
                else:
                    return datetime.strptime(time_str, fmt)
            except:
                continue
    
    return None

camera_start_time = parse_output_txt(OUTPUT_TXT)
print(f"Camera start time: {camera_start_time}")

Camera start time: 2020-09-24 13:09:29


## 3. 載入 Thermal 資料 (多核加速)

In [4]:
def parse_thermal_file(file_path: Path) -> List[Dict]:
    """
    單一檔案解析函數（用於多核處理）
    """
    data = []
    
    try:
        with open(file_path, 'r') as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                
                try:
                    data_dict = json.loads(line)
                    time_parts = data_dict.get('time', [])
                    
                    if len(time_parts) < 6:
                        continue
                    
                    year, month, day, hour, minute, second = time_parts[:6]
                    millisecond = time_parts[7] if len(time_parts) > 7 else 0
                    
                    timestamp = datetime(year, month, day, hour, minute, second, millisecond * 1000)
                    
                    message = data_dict.get('message', [])
                    if len(message) != 768:  # 32*24
                        continue
                    
                    thermal_image = np.array(message, dtype=np.uint8).reshape(24, 32)
                    
                    data.append({
                        'timestamp': timestamp,
                        'image': thermal_image
                    })
                
                except:
                    continue
    
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
    
    return data

# 多核載入
log_files = sorted(THERMAL_DIR.glob('log*.txt'))
print(f"\nFound {len(log_files)} thermal log files")
print(f"Loading with {NUM_WORKERS} workers...")

start_time = time.time()

thermal_data = []
with ProcessPoolExecutor(max_workers=NUM_WORKERS) as executor:
    results = list(tqdm(
        executor.map(parse_thermal_file, log_files),
        total=len(log_files),
        desc="Loading thermal files"
    ))
    
    # 合併結果
    for file_data in results:
        thermal_data.extend(file_data)

# 排序
thermal_data.sort(key=lambda x: x['timestamp'])

elapsed = time.time() - start_time
print(f"\n✓ Loaded {len(thermal_data)} thermal frames in {elapsed:.2f}s")
print(f"  Speed: {len(thermal_data)/elapsed:.0f} frames/sec")

if len(thermal_data) > 0:
    print(f"  Time range: {thermal_data[0]['timestamp']} ~ {thermal_data[-1]['timestamp']}")

if camera_start_time is None:
    camera_start_time = thermal_data[0]['timestamp']
    print(f"  Using thermal first frame as reference")


Found 5 thermal log files
Loading with 3 workers...


Loading thermal files: 100%|██████████| 5/5 [00:00<00:00, 22.14it/s]


✓ Loaded 5318 thermal frames in 0.26s
  Speed: 20679 frames/sec
  Time range: 2020-09-24 13:09:30.268000 ~ 2020-09-24 13:20:52.268000


## 4. 分析 AVI

In [5]:
cap = cv2.VideoCapture(str(AVI_FILE))
if cap.isOpened():
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    duration = frame_count / fps if fps > 0 else 0
    cap.release()
    
    print(f"\n✓ AVI Video:")
    print(f"  Resolution: {width} x {height}")
    print(f"  FPS: {fps}")
    print(f"  Frames: {frame_count}")
    print(f"  Duration: {duration:.2f}s")
else:
    print("⚠️ Cannot open AVI file")
    fps, frame_count, duration = 0, 0, 0


✓ AVI Video:
  Resolution: 640 x 360
  FPS: 24.67
  Frames: 16284
  Duration: 660.07s


## 5. 計算重疊並配對

In [6]:
# 重疊區間
thermal_start = thermal_data[0]['timestamp']
thermal_end = thermal_data[-1]['timestamp']
rgb_start = camera_start_time
rgb_end = rgb_start + timedelta(seconds=duration)

overlap_start = max(thermal_start, rgb_start)
overlap_end = min(thermal_end, rgb_end)

print(f"\nTime Sync:")
print(f"  Overlap: {overlap_start} ~ {overlap_end}")
print(f"  Duration: {(overlap_end - overlap_start).total_seconds():.2f}s")

# 創建時間軸
target_timeline = []
current = overlap_start
interval = timedelta(seconds=FRAME_INTERVAL)

while current <= overlap_end:
    target_timeline.append(current)
    current += interval

print(f"  Target frames: {len(target_timeline)} @ {TARGET_FPS} fps")

# 配對函數
def find_nearest_thermal(target_time, thermal_data, tolerance=TIMESTAMP_TOLERANCE):
    thermal_times = [d['timestamp'] for d in thermal_data]
    idx = np.searchsorted(thermal_times, target_time)
    
    best_idx = None
    best_diff = float('inf')
    
    for cand_idx in [idx-1, idx]:
        if 0 <= cand_idx < len(thermal_times):
            diff = abs((thermal_times[cand_idx] - target_time).total_seconds())
            if diff < tolerance and diff < best_diff:
                best_idx = cand_idx
                best_diff = diff
    
    return thermal_data[best_idx] if best_idx is not None else None

def find_nearest_rgb_frame(target_time, video_start, fps, frame_count, tolerance=TIMESTAMP_TOLERANCE):
    target_offset = (target_time - video_start).total_seconds()
    
    if target_offset < 0 or target_offset > (frame_count / fps):
        return None, float('inf')
    
    frame_idx = int(round(target_offset * fps))
    frame_idx = max(0, min(frame_idx, frame_count - 1))
    
    actual_offset = frame_idx / fps
    error = abs(actual_offset - target_offset)
    
    return (frame_idx, error) if error <= tolerance else (None, float('inf'))

# 配對
print("\nMatching frames...")
pairs = []

for target_time in tqdm(target_timeline, desc="Pairing"):
    thermal_frame = find_nearest_thermal(target_time, thermal_data)
    if thermal_frame is None:
        continue
    
    rgb_idx, error = find_nearest_rgb_frame(target_time, camera_start_time, fps, frame_count)
    if rgb_idx is None:
        continue
    
    pairs.append({
        'timestamp': target_time,
        'thermal': thermal_frame,
        'rgb_frame_idx': rgb_idx,
        'thermal_error': abs((thermal_frame['timestamp'] - target_time).total_seconds()),
        'rgb_error': error
    })

print(f"\n✓ Paired {len(pairs)} frames ({len(pairs)/len(target_timeline)*100:.1f}% success)")


Time Sync:
  Overlap: 2020-09-24 13:09:30.268000 ~ 2020-09-24 13:20:29.072963
  Duration: 658.80s
  Target frames: 5271 @ 8 fps

Matching frames...


Pairing: 100%|██████████| 5271/5271 [00:31<00:00, 169.35it/s]


✓ Paired 659 frames (12.5% success)


## 6. 提取 RGB 並儲存 (多核加速)

In [7]:
def process_pair_batch(args):
    """
    批次處理配對（減少影片重複開啟）
    """
    batch_pairs, avi_path, flip_thermal, flip_rgb, sync_dir, export_dir = args
    
    cap = cv2.VideoCapture(str(avi_path))
    if not cap.isOpened():
        return []
    
    results = []
    
    # 先排序（減少 seek 次數）
    batch_pairs_sorted = sorted(batch_pairs, key=lambda x: x[1]['rgb_frame_idx'])
    
    current_frame = 0
    frame_cache = {}
    
    for pair_idx, pair in batch_pairs_sorted:
        target_frame_idx = pair['rgb_frame_idx']
        
        # 載入幀
        if target_frame_idx not in frame_cache:
            if target_frame_idx < current_frame:
                cap.set(cv2.CAP_PROP_POS_FRAMES, target_frame_idx)
                current_frame = target_frame_idx
            
            ret = True
            while current_frame <= target_frame_idx and ret:
                ret, frame = cap.read()
                current_frame += 1
            
            if ret:
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame_cache[target_frame_idx] = frame_rgb
        
        if target_frame_idx in frame_cache:
            rgb_frame = frame_cache[target_frame_idx]
            thermal_img = pair['thermal']['image']
            
            # 翻轉
            if flip_thermal:
                thermal_img = cv2.flip(thermal_img, 1)
            if flip_rgb:
                rgb_frame = cv2.flip(rgb_frame, 1)
            
            pair_name = f"pair_{pair_idx:05d}"
            
            # 儲存
            np.save(sync_dir / 'thermal' / f"{pair_name}_thermal.npy", thermal_img)
            np.save(sync_dir / 'rgb' / f"{pair_name}_rgb.npy", rgb_frame)
            
            rgb_jpg_path = export_dir / 'rgb' / f"{pair_name}.jpg"
            cv2.imwrite(str(rgb_jpg_path), cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2BGR))
            
            thermal_jpg_path = export_dir / 'thermal' / f"{pair_name}.jpg"
            cv2.imwrite(str(thermal_jpg_path), thermal_img)
            
            results.append({
                'pair_id': pair_name,
                'pair_index': pair_idx,
                'timestamp': pair['timestamp'],
                'rgb_frame_idx': pair['rgb_frame_idx'],
                'thermal_error_ms': pair['thermal_error'] * 1000,
                'rgb_error_ms': pair['rgb_error'] * 1000,
                'rgb_jpg_path': str(rgb_jpg_path.relative_to(export_dir)),
                'thermal_jpg_path': str(thermal_jpg_path.relative_to(export_dir))
            })
    
    cap.release()
    return results

# 分批處理（每個 worker 獨立開啟影片）
print(f"\nExtracting RGB frames with {NUM_WORKERS} workers...")
start_time = time.time()

# 將 pairs 分成批次
batches = []
for i in range(0, len(pairs), CHUNK_SIZE):
    batch = [(idx, pair) for idx, pair in enumerate(pairs[i:i+CHUNK_SIZE], start=i)]
    batches.append((
        batch,
        AVI_FILE,
        FLIP_THERMAL,
        FLIP_RGB,
        SYNCHRONIZED_DIR,
        RGB_EXPORT_DIR
    ))

# 多核處理
metadata_list = []
with ProcessPoolExecutor(max_workers=NUM_WORKERS) as executor:
    batch_results = list(tqdm(
        executor.map(process_pair_batch, batches),
        total=len(batches),
        desc="Extracting"
    ))
    
    for batch_result in batch_results:
        metadata_list.extend(batch_result)

elapsed = time.time() - start_time
print(f"\n✓ Extracted {len(metadata_list)} pairs in {elapsed:.2f}s")
print(f"  Speed: {len(metadata_list)/elapsed:.1f} pairs/sec")

# 儲存 metadata
metadata_df = pd.DataFrame(metadata_list)
metadata_df = metadata_df.sort_values('pair_index').reset_index(drop=True)
metadata_df.to_csv(SYNCHRONIZED_DIR / 'metadata.csv', index=False)
metadata_df.to_csv(RGB_EXPORT_DIR / 'labeling_metadata.csv', index=False)

print(f"\n✓ Saved metadata to:")
print(f"  {SYNCHRONIZED_DIR / 'metadata.csv'}")
print(f"  {RGB_EXPORT_DIR / 'labeling_metadata.csv'}")


Extracting RGB frames with 3 workers...


Extracting: 100%|██████████| 7/7 [00:18<00:00,  2.58s/it]


✓ Extracted 659 pairs in 18.08s
  Speed: 36.4 pairs/sec

✓ Saved metadata to:
  output/synchronized_final/metadata.csv
  output/rgb_for_labeling/labeling_metadata.csv


## 7. 生成標記範本

In [9]:
# 針對人體姿勢的標記範本
label_template = metadata_df[['pair_id', 'pair_index', 'rgb_jpg_path']].copy()
label_template['pose'] = ''  # 姿勢: lying/sitting/fallen
label_template['person_visible'] = ''  # 人是否可見: yes/no/partial
label_template['confidence'] = ''  # 1-5
label_template['notes'] = ''

label_template.to_csv(RGB_EXPORT_DIR / 'pose_labels_template.csv', index=False)

# 姿勢標記說明
readme = f"""# 人體姿勢標記指南

## 目標
為病床上的人標記姿勢，用於訓練熱成像超解析度模型。

## 姿勢類別
1. **lying** (躺著) - 人平躺在床上
2. **sitting** (坐著) - 人坐在床上或床邊
3. **fallen** (跌倒) - 人從床上跌落
4. **none** (無人) - 床上沒有人

## 標記欄位說明

### pose (必填)
- 填入: `lying`, `sitting`, `fallen`, 或 `none`

### person_visible (必填)
- `yes` - 人清楚可見
- `partial` - 人部分可見（被遮擋或在畫面邊緣）
- `no` - 無法看到人

### confidence (選填)
- 1 = 很不確定
- 2 = 不太確定
- 3 = 普通
- 4 = 確定
- 5 = 非常確定

### notes (選填)
- 任何備註（例如：「被棉被遮擋」、「畫面模糊」）

## 標記範例

| pair_id | rgb_jpg_path | pose | person_visible | confidence | notes |
|---------|--------------|------|----------------|------------|-------|
| pair_00000 | rgb/pair_00000.jpg | lying | yes | 5 | 清楚躺著 |
| pair_00001 | rgb/pair_00001.jpg | sitting | yes | 4 | 坐在床邊 |
| pair_00002 | rgb/pair_00002.jpg | lying | partial | 3 | 被棉被部分遮擋 |
| pair_00003 | rgb/pair_00003.jpg | fallen | yes | 5 | 明顯跌下床 |
| pair_00004 | rgb/pair_00004.jpg | none | no | 5 | 床上無人 |

## 標記流程

1. 開啟 `pose_labels_template.csv`
2. 查看 `rgb/*.jpg` 圖片
3. 可參考對應的 `thermal/*.jpg` 確認
4. 填寫 pose 和 person_visible 欄位
5. (可選) 填寫 confidence 和 notes
6. 另存為 `pose_labels.csv`

## 注意事項

- 總共 **{len(metadata_df)}** 張圖片需要標記
- 如果不確定，請在 notes 欄位說明
- 標記完成後，將 `pose_labels.csv` 放回此目錄

## 後續用途

您的標記將用於：
1. 訓練 RGB 人體分割模型
2. 過濾高品質配對（person_visible=yes）
3. 評估超解析度在不同姿勢的表現
4. 訓練姿勢辨識模型
"""

with open(RGB_EXPORT_DIR / 'POSE_LABELING_README.md', 'w', encoding='utf-8') as f:
    f.write(readme)

print(f"\n✓ Pose labeling template created:")
print(f"  {RGB_EXPORT_DIR / 'pose_labels_template.csv'}")
print(f"  {RGB_EXPORT_DIR / 'POSE_LABELING_README.md'}")


✓ Pose labeling template created:
  output/rgb_for_labeling/pose_labels_template.csv
  output/rgb_for_labeling/POSE_LABELING_README.md


In [ ]:
# 診斷哪些時間點配對失敗

def diagnose_pairing(target_timeline, thermal_data, camera_start_time, fps, frame_count):
    failed_matches = {
        'no_thermal': [],
        'no_rgb': [],
        'high_error': []
    }
    
    for idx, target_time in enumerate(target_timeline):
        # 1. 檢查 Thermal
        thermal_frame = find_nearest_thermal(target_time, thermal_data)
        if thermal_frame is None:
            failed_matches['no_thermal'].append(idx)
            continue
        
        # 2. 檢查 RGB
        rgb_idx, error = find_nearest_rgb_frame(
            target_time, 
            camera_start_time, 
            fps, 
            frame_count,
            tolerance=TIMESTAMP_TOLERANCE
        )
        
        if rgb_idx is None:
            if error == float('inf'):
                failed_matches['no_rgb'].append(idx)
            else:
                failed_matches['high_error'].append((idx, error * 1000))  # ms
    
    return failed_matches

# 診斷
failed = diagnose_pairing(target_timeline, thermal_data, camera_start_time, fps, frame_count)

print("Pairing failure analysis:")
print(f"  No thermal match: {len(failed['no_thermal'])}")
print(f"  No RGB frame: {len(failed['no_rgb'])}")
print(f"  Error > tolerance: {len(failed['high_error'])}")

if failed['high_error']:
    errors = [e for _, e in failed['high_error']]
    print(f"\nError distribution (ms):")
    print(f"  Max: {max(errors):.2f}")
    print(f"  Mean: {np.mean(errors):.2f}")
    print(f"  Samples: {errors[:5]}")

## 總結

In [10]:
print("\n" + "="*70)
print("資料準備完成 - 針對人體姿勢辨識")
print("="*70)

print(f"\n1. 時間對齊")
print(f"   Camera start: {camera_start_time}")
print(f"   Pairs: {len(metadata_df)}")
print(f"   Avg thermal error: {metadata_df['thermal_error_ms'].mean():.2f}ms")
print(f"   Avg RGB error: {metadata_df['rgb_error_ms'].mean():.2f}ms")

print(f"\n2. 多核加速效能")
print(f"   CPU cores used: {NUM_WORKERS}")
print(f"   Total processing time: {elapsed:.2f}s")
print(f"   Speed: {len(metadata_df)/elapsed:.1f} pairs/sec")

print(f"\n3. 輸出檔案")
print(f"   訓練資料: {SYNCHRONIZED_DIR}")
print(f"   標記用圖片: {RGB_EXPORT_DIR}")

print(f"\n4. 下一步")
print(f"   📝 標記姿勢:")
print(f"      1. 閱讀 {RGB_EXPORT_DIR / 'POSE_LABELING_README.md'}")
print(f"      2. 填寫 pose_labels_template.csv")
print(f"      3. 另存為 pose_labels.csv")

print(f"\n   🚀 標記完成後:")
print(f"      我會給您完整的訓練 pipeline:")
print(f"      1. RGB 人體分割模型")
print(f"      2. Semantic-Guided Thermal SR")
print(f"      3. 姿勢辨識模型")

print("\n" + "="*70)


資料準備完成 - 針對人體姿勢辨識

1. 時間對齊
   Camera start: 2020-09-24 13:09:29
   Pairs: 659
   Avg thermal error: 0.00ms
   Avg RGB error: 10.10ms

2. 多核加速效能
   CPU cores used: 3
   Total processing time: 18.08s
   Speed: 36.4 pairs/sec

3. 輸出檔案
   訓練資料: output/synchronized_final
   標記用圖片: output/rgb_for_labeling

4. 下一步
   📝 標記姿勢:
      1. 閱讀 output/rgb_for_labeling/POSE_LABELING_README.md
      2. 填寫 pose_labels_template.csv
      3. 另存為 pose_labels.csv

   🚀 標記完成後:
      我會給您完整的訓練 pipeline:
      1. RGB 人體分割模型
      2. Semantic-Guided Thermal SR
      3. 姿勢辨識模型

